In [ ]:
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Dense, Activation , LSTM , Dropout , AveragePooling3D
import pandas as pd

import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv('features.csv')
df.head()

,Age,male,female,other,ROSC,OHCA,vfib,TTM,Unnamed: 8,Unnamed: 9,...,Unnamed: 108,Unnamed: 109,Unnamed: 110,Unnamed: 111,Unnamed: 112,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116,CPC
0,53.0,1,0,0,NaN,1.0,1.0,33.0,-0.000659,0.000224,...,0.038520,0.077394,0.062352,0.07532,0.066819,0.047992,0.060609,0.045147,1.000,1
1,85.0,0,1,0,7.0,0.0,0.0,NaN,-0.000428,-0.004372,...,0.067099,0.061575,0.193790,0.16430,0.049054,0.063502,0.141550,0.127730,1.000,1
2,48.0,1,0,0,NaN,1.0,1.0,36.0,-0.045466,-0.015661,...,0.331930,0.373830,0.567240,0.74219,0.354900,0.348280,0.312040,0.454560,0.983,1
3,45.0,1,0,0,NaN,1.0,1.0,33.0,-0.001706,-0.003433,...,0.383510,0.035878,0.266470,0.46054,3.725300,3.699300,0.536280,0.461510,1.000,1
4,51.0,1,0,0,24.0,1.0,1.0,33.0,0.002509,0.001997,...,0.073369,0.053554,0.090785,0.13919,0.084864,0.064359,0.123880,0.237830,1.000,1


In [ ]:
df['ROSC'].corr(df['CPC'])


0.13709439354580877

In [ ]:
print(df["ROSC"].min())
df["ROSC"].max()

1.0


111.0

In [ ]:
df['TTM'] = df['TTM'].fillna(df['TTM'].mode()[0]) #Filling NULL values of TTM column with mode

df.drop(['OHCA'], axis=1,inplace=True)

#Filling NULL values of vfib column
import math
list1=[3,5]
for i in range(len(df)):
    #print(df.loc[i, "Name"], df.loc[i, "Age"])
    if(math.isnan(df.loc[i,"vfib"])):
      if(df.loc[i,"CPC"] in list1):
        #replace with 0
        df.loc[i,"vfib"]=0
      else:
       #replace with 1
       df.loc[i,"vfib"]=1



In [ ]:
df

,Age,male,female,other,ROSC,vfib,TTM,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 108,Unnamed: 109,Unnamed: 110,Unnamed: 111,Unnamed: 112,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116,CPC
0,53.0,1,0,0,NaN,1.0,33.0,-0.000659,0.000224,0.000740,...,0.038520,0.077394,0.062352,0.075320,0.066819,0.047992,0.060609,0.045147,1.000,1
1,85.0,0,1,0,7.0,0.0,33.0,-0.000428,-0.004372,0.002532,...,0.067099,0.061575,0.193790,0.164300,0.049054,0.063502,0.141550,0.127730,1.000,1
2,48.0,1,0,0,NaN,1.0,36.0,-0.045466,-0.015661,-0.010341,...,0.331930,0.373830,0.567240,0.742190,0.354900,0.348280,0.312040,0.454560,0.983,1
3,45.0,1,0,0,NaN,1.0,33.0,-0.001706,-0.003433,-0.001583,...,0.383510,0.035878,0.266470,0.460540,3.725300,3.699300,0.536280,0.461510,1.000,1
4,51.0,1,0,0,24.0,1.0,33.0,0.002509,0.001997,0.000132,...,0.073369,0.053554,0.090785,0.139190,0.084864,0.064359,0.123880,0.237830,1.000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,87.0,1,0,0,7.0,0.0,33.0,-0.009449,0.009449,-0.004638,...,0.981820,0.981820,0.654500,1.652000,0.564080,1.536100,12.702000,17.269000,1.000,5
603,26.0,1,0,0,52.0,0.0,33.0,-0.000501,0.000673,0.000666,...,0.006795,0.005688,0.006546,0.007372,0.008170,0.007294,0.007106,0.010509,0.933,5
604,63.0,1,0,0,NaN,1.0,36.0,-0.003325,0.003325,-0.001804,...,0.010443,0.010443,0.011846,0.017321,0.007972,0.008383,0.031595,0.022916,1.000,5
605,72.0,1,0,0,NaN,1.0,36.0,-0.002634,-0.000994,0.007060,...,0.246110,0.180780,0.243380,0.332630,0.161790,0.164970,0.375060,0.294810,1.000,1


In [ ]:
test_data = df[df['ROSC'].isnull()] # test_data are the rows of the dataframe where ROSC is NULL
test_data
X_test=test_data.drop('ROSC',axis='columns') # X_test consists of all features of test_data except ROSC


In [ ]:
X_test

,Age,male,female,other,vfib,TTM,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,...,Unnamed: 108,Unnamed: 109,Unnamed: 110,Unnamed: 111,Unnamed: 112,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116,CPC
0,53.0,1,0,0,1.0,33.0,-0.000659,0.000224,0.000740,-0.002887,...,0.038520,0.077394,0.062352,0.075320,0.066819,0.047992,0.060609,0.045147,1.000,1
2,48.0,1,0,0,1.0,36.0,-0.045466,-0.015661,-0.010341,0.000420,...,0.331930,0.373830,0.567240,0.742190,0.354900,0.348280,0.312040,0.454560,0.983,1
3,45.0,1,0,0,1.0,33.0,-0.001706,-0.003433,-0.001583,0.002543,...,0.383510,0.035878,0.266470,0.460540,3.725300,3.699300,0.536280,0.461510,1.000,1
6,39.0,0,1,0,1.0,33.0,-0.001550,-0.000120,-0.001135,-0.002068,...,0.418200,0.538690,0.722080,0.638700,0.966420,0.360940,0.250780,0.730940,1.000,2
7,51.0,1,0,0,1.0,33.0,0.002022,0.001432,0.001003,-0.002236,...,0.388090,0.230890,3.160500,0.566640,0.220150,0.256340,0.986220,0.659780,1.000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599,68.0,1,0,0,1.0,33.0,-0.003002,0.000110,0.000779,0.000946,...,0.268880,0.119270,0.349090,0.230460,0.140810,0.126550,0.219680,0.176350,1.000,2
601,63.0,0,1,0,1.0,36.0,-0.001268,-0.001446,0.000885,0.002788,...,0.037248,0.071396,0.045025,0.031271,0.023467,0.049387,0.078017,0.055949,1.000,2
604,63.0,1,0,0,1.0,36.0,-0.003325,0.003325,-0.001804,-0.000759,...,0.010443,0.010443,0.011846,0.017321,0.007972,0.008383,0.031595,0.022916,1.000,5
605,72.0,1,0,0,1.0,36.0,-0.002634,-0.000994,0.007060,0.014736,...,0.246110,0.180780,0.243380,0.332630,0.161790,0.164970,0.375060,0.294810,1.000,1


In [ ]:
X_test.isnull().sum().sum()
X_test = X_test.dropna()

In [ ]:
train_data = df.dropna(subset=['ROSC']) # train_data are the rows of the dataframe where ROSC is not null
train_data
Y_train=train_data['ROSC'].to_frame()
Y_train

,ROSC
1,7.0
4,24.0
5,20.0
8,3.0
9,37.0
...,...
597,5.0
598,15.0
600,42.0
602,7.0


In [ ]:
X_train=train_data.drop('ROSC',axis=1)
X_train

,Age,male,female,other,vfib,TTM,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,...,Unnamed: 108,Unnamed: 109,Unnamed: 110,Unnamed: 111,Unnamed: 112,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116,CPC
1,85.0,0,1,0,0.0,33.0,-0.000428,-0.004372,0.002532,-0.001421,...,0.067099,0.061575,0.193790,0.164300,0.049054,0.063502,0.141550,0.127730,1.000,1
4,51.0,1,0,0,1.0,33.0,0.002509,0.001997,0.000132,-0.000590,...,0.073369,0.053554,0.090785,0.139190,0.084864,0.064359,0.123880,0.237830,1.000,1
5,73.0,1,0,0,0.0,33.0,0.002957,-0.002957,0.001360,0.002903,...,0.016262,0.016262,0.017615,0.042830,0.010897,0.030636,0.030478,0.027007,1.000,1
8,56.0,1,0,0,0.0,33.0,-0.007959,-0.003433,-0.001291,-0.001032,...,0.391970,0.093881,2.264000,1.929500,0.460540,0.194370,0.393880,0.381230,0.983,1
9,41.0,1,0,0,1.0,33.0,0.079634,-0.005835,-0.020240,-0.048490,...,0.056507,0.061195,0.048762,0.065993,0.067331,0.064432,192.350000,189.660000,1.000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597,90.0,1,0,0,0.0,33.0,0.001235,-0.001235,-0.001020,-0.000750,...,0.102900,0.102900,0.208510,0.076433,0.019873,0.036233,0.560510,0.478890,1.000,5
598,78.0,1,0,0,0.0,33.0,-0.000376,-0.000898,-0.002892,-0.003497,...,0.043954,0.026698,1.550800,0.443030,0.042389,0.041904,0.330650,0.053351,1.000,4
600,37.0,0,1,0,1.0,33.0,0.000736,0.000684,0.000117,-0.000329,...,0.005276,0.005278,0.007224,0.005173,0.005268,0.005323,0.007147,0.006355,0.833,5
602,87.0,1,0,0,0.0,33.0,-0.009449,0.009449,-0.004638,-0.000742,...,0.981820,0.981820,0.654500,1.652000,0.564080,1.536100,12.702000,17.269000,1.000,5


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [ ]:
names = ["Nearest Neighbors", "Linear SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "AdaBoost",
         "Naive Bayes"]

In [ ]:
classifiers = [
    KNeighborsClassifier(5),
    SVC(kernel="linear", C=0.025),
    GaussianProcessClassifier(1.0 * RBF(1.0), warm_start=True),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    AdaBoostClassifier(),
    ]


In [ ]:
# Using MLR
predictions=[]
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
clf_score=[] #scores for each of the classifiers
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for name, clf in zip(names, classifiers):
        clf.fit(X_train, Y_train)
        predictions.append(clf.predict(X_test)) #For each ML model, it predicts the ROSC values that are NULL (corresponding to X_test)

In [ ]:
df.dropna(subset=['Age'], inplace=True)

df

,Age,male,female,other,ROSC,vfib,TTM,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 108,Unnamed: 109,Unnamed: 110,Unnamed: 111,Unnamed: 112,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116,CPC
0,53.0,1,0,0,NaN,1.0,33.0,-0.000659,0.000224,0.000740,...,0.038520,0.077394,0.062352,0.075320,0.066819,0.047992,0.060609,0.045147,1.000,1
1,85.0,0,1,0,7.0,0.0,33.0,-0.000428,-0.004372,0.002532,...,0.067099,0.061575,0.193790,0.164300,0.049054,0.063502,0.141550,0.127730,1.000,1
2,48.0,1,0,0,NaN,1.0,36.0,-0.045466,-0.015661,-0.010341,...,0.331930,0.373830,0.567240,0.742190,0.354900,0.348280,0.312040,0.454560,0.983,1
3,45.0,1,0,0,NaN,1.0,33.0,-0.001706,-0.003433,-0.001583,...,0.383510,0.035878,0.266470,0.460540,3.725300,3.699300,0.536280,0.461510,1.000,1
4,51.0,1,0,0,24.0,1.0,33.0,0.002509,0.001997,0.000132,...,0.073369,0.053554,0.090785,0.139190,0.084864,0.064359,0.123880,0.237830,1.000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,87.0,1,0,0,7.0,0.0,33.0,-0.009449,0.009449,-0.004638,...,0.981820,0.981820,0.654500,1.652000,0.564080,1.536100,12.702000,17.269000,1.000,5
603,26.0,1,0,0,52.0,0.0,33.0,-0.000501,0.000673,0.000666,...,0.006795,0.005688,0.006546,0.007372,0.008170,0.007294,0.007106,0.010509,0.933,5
604,63.0,1,0,0,NaN,1.0,36.0,-0.003325,0.003325,-0.001804,...,0.010443,0.010443,0.011846,0.017321,0.007972,0.008383,0.031595,0.022916,1.000,5
605,72.0,1,0,0,NaN,1.0,36.0,-0.002634,-0.000994,0.007060,...,0.246110,0.180780,0.243380,0.332630,0.161790,0.164970,0.375060,0.294810,1.000,1


In [ ]:
new_df = df.copy()


In [ ]:
#Updating the missing values of ROSC column in the dataframe
i=0
for index, value in new_df[new_df['ROSC'].isnull()].iterrows():
    new_df.at[index, 'ROSC'] = predictions[1][i]
    i+=1

In [ ]:
new_df

,Age,male,female,other,ROSC,vfib,TTM,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 108,Unnamed: 109,Unnamed: 110,Unnamed: 111,Unnamed: 112,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116,CPC
0,53.0,1,0,0,18.0,1.0,33.0,-0.000659,0.000224,0.000740,...,0.038520,0.077394,0.062352,0.075320,0.066819,0.047992,0.060609,0.045147,1.000,1
1,85.0,0,1,0,7.0,0.0,33.0,-0.000428,-0.004372,0.002532,...,0.067099,0.061575,0.193790,0.164300,0.049054,0.063502,0.141550,0.127730,1.000,1
2,48.0,1,0,0,14.0,1.0,36.0,-0.045466,-0.015661,-0.010341,...,0.331930,0.373830,0.567240,0.742190,0.354900,0.348280,0.312040,0.454560,0.983,1
3,45.0,1,0,0,13.0,1.0,33.0,-0.001706,-0.003433,-0.001583,...,0.383510,0.035878,0.266470,0.460540,3.725300,3.699300,0.536280,0.461510,1.000,1
4,51.0,1,0,0,24.0,1.0,33.0,0.002509,0.001997,0.000132,...,0.073369,0.053554,0.090785,0.139190,0.084864,0.064359,0.123880,0.237830,1.000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,87.0,1,0,0,7.0,0.0,33.0,-0.009449,0.009449,-0.004638,...,0.981820,0.981820,0.654500,1.652000,0.564080,1.536100,12.702000,17.269000,1.000,5
603,26.0,1,0,0,52.0,0.0,33.0,-0.000501,0.000673,0.000666,...,0.006795,0.005688,0.006546,0.007372,0.008170,0.007294,0.007106,0.010509,0.933,5
604,63.0,1,0,0,15.0,1.0,36.0,-0.003325,0.003325,-0.001804,...,0.010443,0.010443,0.011846,0.017321,0.007972,0.008383,0.031595,0.022916,1.000,5
605,72.0,1,0,0,15.0,1.0,36.0,-0.002634,-0.000994,0.007060,...,0.246110,0.180780,0.243380,0.332630,0.161790,0.164970,0.375060,0.294810,1.000,1


In [ ]:
new_df.isnull().sum().sum()

0

In [ ]:
new_df.to_csv('features_svc.csv', index=False)